In [2]:
from dotenv import load_dotenv
import os
load_dotenv(override=True)  
api_key = os.getenv("GOOGLE_API_KEY")

In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(google_api_key=api_key, model="gemini-2.0-flash", temperature=0.2)
poem = llm.invoke("write a poem on my love for dosa").content.strip()
print(poem)

In [37]:
from langchain_community.utilities import SQLDatabase
from langchain_experimental.sql import SQLDatabaseChain
from sqlalchemy import create_engine

In [ ]:
class CleanSQLDatabase(SQLDatabase):
    def run(self, command: str, fetch: str = "all"):
        # strip markdown fences before executing
        clean_command = (
            command.replace("```sql", "")
                   .replace("```", "")
                   .strip()
        )
        print("Executing Clean SQL:", clean_command)  # debug
        return super().run(clean_command, fetch=fetch)

In [44]:

db_user = "root"
db_password = "root"
db_host = "localhost"
db_name = "atliq_tshirts"

engine = create_engine(f"mysql+pymysql://{db_user}:{db_password}@{db_host}/{db_name}")
db = CleanSQLDatabase(engine)

# ALTER USER 'root'@'localhost' IDENTIFIED WITH mysql_native_password BY 'root'


In [36]:

# from langchain.prompts.prompt import PromptTemplate

# # Custom prompt: forces plain SQL only
# _SQL_PROMPT = """Given an input question, create a syntactically correct {dialect} query to run.
# Do not include ``` or any markdown formatting.
# Only return the SQL query.

# Question: {input}
# SQLQuery:"""

# CUSTOM_PROMPT = PromptTemplate(
#     input_variables=["input", "dialect"],
#     template=_SQL_PROMPT,
# )

In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI
llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash", temperature=0)

db_chain = SQLDatabaseChain.from_llm(llm,db,verbose=True,use_query_checker=True,return_direct=True)
question = "How many t-shirts do we have left for nike in extra small size and white color?"

try:
    result = db_chain.invoke(question)
    print("Final Answer:", result)
except Exception as e:
    print("Error:", e)

In [ ]:
db_chain = SQLDatabaseChain.from_llm(llm, db, verbose=True, use_query_checker=True, return_direct=True)
qns1 = "How many t-shirts do we have left for nike in extra small size and white color?"
db_chain.invoke(qns1)

In [ ]:
qns2 = "How much is the price of the inventory for all small size t-shirts?"
result = db_chain.invoke(qns2)
print("Final Answer:", result)

In [ ]:
qns2 = "SELECT SUM(price*stock_quantity) FROM t_shirts WHERE size = 'S'"
result = db_chain.invoke(qns2)
print("Final Answer:", result)

In [ ]:
qns3 = db_chain.invoke("If we have to sell all the Levi’s T-shirts today with discounts applied. How much revenue our store will generate (post discounts)?")

In [ ]:
sql_code = "select sum(a.total_amount * ((100-COALESCE(discounts.pct_discount,0))/100)) as total_revenue from (select sum(price*stock_quantity) as total_amount, t_shirt_id from t_shirts where brand = 'Levi' group by t_shirt_id) a left join discounts on a.t_shirt_id = discounts.t_shirt_id"
 

qns3 = db_chain.invoke(sql_code)

In [ ]:
qns4 = db_chain.invoke("SELECT SUM(price * stock_quantity) FROM t_shirts WHERE brand = 'Levi'")

In [ ]:
qns5 = db_chain.invoke("How many white color Levi's t shirts we have available?")

In [ ]:
qns5 = "SELECT sum(stock_quantity) FROM t_shirts WHERE brand = 'Levi' AND color = 'White'"
result = db_chain.invoke(qns5)
print("Final Answer:", result)

In [66]:
few_shots = [
    {'Question' : "How many t-shirts do we have left for Nike in XS size and white color?",
     'SQLQuery' : "SELECT sum(stock_quantity) FROM t_shirts WHERE brand = 'Nike' AND color = 'White' AND size = 'XS'",
     'SQLResult': "Result of the SQL query",
     'Answer' : qns1},
    {'Question': "How much is the total price of the inventory for all S-size t-shirts?",
     'SQLQuery':"SELECT SUM(price*stock_quantity) FROM t_shirts WHERE size = 'S'",
     'SQLResult': "Result of the SQL query",
     'Answer': qns2},
    {'Question': "If we have to sell all the Levi’s T-shirts today with discounts applied. How much revenue  our store will generate (post discounts)?" ,
     'SQLQuery' : """SELECT sum(a.total_amount * ((100-COALESCE(discounts.pct_discount,0))/100)) as total_revenue from
(select sum(price*stock_quantity) as total_amount, t_shirt_id from t_shirts where brand = 'Levi'
group by t_shirt_id) a left join discounts on a.t_shirt_id = discounts.t_shirt_id
 """,
     'SQLResult': "Result of the SQL query",
     'Answer': qns3} ,
     {'Question' : "If we have to sell all the Levi’s T-shirts today. How much revenue our store will generate without discount?" ,
      'SQLQuery': "SELECT SUM(price * stock_quantity) FROM t_shirts WHERE brand = 'Levi'",
      'SQLResult': "Result of the SQL query",
      'Answer' : qns4},
    {'Question': "How many white color Levi's shirt I have?",
     'SQLQuery' : "SELECT sum(stock_quantity) FROM t_shirts WHERE brand = 'Levi' AND color = 'White'",
     'SQLResult': "Result of the SQL query",
     'Answer' : qns5
     }
]

In [71]:
from langchain.prompts import SemanticSimilarityExampleSelector
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma


embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')

# to_vectorize = [" ".join(example.values()) for example in few_shots]
to_vectorize = [
    " ".join(str(v) for v in example.values())  # cast each value to str
    for example in few_shots
]


In [ ]:
from langchain.vectorstores import Chroma
vectorstore = Chroma.from_texts(to_vectorize, embedding=embeddings, metadatas=few_shots)

In [ ]:
example_selector = SemanticSimilarityExampleSelector(
    vectorstore=vectorstore,
    k=2,
)

example_selector.select_examples({"Question": "How many Adidas T shirts I have left in my store?"})

In [78]:
### my sql based instruction prompt
mysql_prompt = """You are a MySQL expert. Given an input question, first create a syntactically correct MySQL query to run, then look at the results of the query and return the answer to the input question.
Unless the user specifies in the question a specific number of examples to obtain, query for at most {top_k} results using the LIMIT clause as per MySQL. You can order the results to return the most informative data in the database.
Never query for all columns from a table. You must query only the columns that are needed to answer the question. Wrap each column name in backticks (`) to denote them as delimited identifiers.
Pay attention to use only the column names you can see in the tables below. Be careful to not query for columns that do not exist. Also, pay attention to which column is in which table.
Pay attention to use CURDATE() function to get the current date, if the question involves "today".

Use the following format:

Question: Question here
SQLQuery: Query to run with no pre-amble
SQLResult: Result of the SQLQuery
Answer: Final answer here

No pre-amble.
"""

In [ ]:
from langchain.prompts import FewShotPromptTemplate
from langchain.chains.sql_database.prompt import PROMPT_SUFFIX, _mysql_prompt

print(PROMPT_SUFFIX)

In [74]:
from langchain.prompts.prompt import PromptTemplate

example_prompt = PromptTemplate(
    input_variables=["Question", "SQLQuery", "SQLResult","Answer",],
    template="\nQuestion: {Question}\nSQLQuery: {SQLQuery}\nSQLResult: {SQLResult}\nAnswer: {Answer}",
)

In [ ]:
print(_mysql_prompt)

In [ ]:
few_shot_prompt = FewShotPromptTemplate(
    example_selector=example_selector,
    example_prompt=example_prompt,
    prefix=mysql_prompt,
    suffix=PROMPT_SUFFIX,
    input_variables=["input", "table_info", "top_k"], #These variables are used in the prefix and suffix
)

In [ ]:
new_chain = SQLDatabaseChain.from_llm(llm, db, verbose=True, prompt=few_shot_prompt)

In [ ]:
new_chain("How many white color Levi's shirt I have?")

In [ ]:
new_chain("How much is the price of the inventory for all small size t-shirts?")

In [ ]:
new_chain("How much is the price of all white color levi t shirts?")

In [ ]:
new_chain("If we have to sell all the Nike’s T-shirts today with discounts applied. How much revenue  our store will generate (post discounts)?")

In [ ]:
new_chain("If we have to sell all the Van Heuson T-shirts today with discounts applied. How much revenue  our store will generate (post discounts)?")

In [ ]:
new_chain.run('How much revenue  our store will generate by selling all Van Heuson TShirts without discount?')